### `Q-1:` You are given a Multi index DataFrame. You task is to swap level-0 index with level-0 column.
* Change Branch -> ('cse', 'ece') as columns at level-0 and ('delhi'	'mumbai') as level-0 Index
* And Sort on row index level-0

In [2]:
# Given Code Snippets
import pandas as pd
import numpy as np
index_val = [('cse',2019),('cse',2020),('cse',2021),('cse',2022),('ece',2019),('ece',2020),('ece',2021),('ece',2022)]
multiindex = pd.MultiIndex.from_tuples(index_val)
df = pd.DataFrame(
    [
        [1,2,0,0],
        [3,4,0,0],
        [5,6,0,0],
        [7,8,0,0],
        [9,10,0,0],
        [11,12,0,0],
        [13,14,0,0],
        [15,16,0,0],
    ],
    index = multiindex,
    columns = pd.MultiIndex.from_product([['delhi','mumbai'],['avg_package','students']])
)

df

delhi               mumbai         
         avg_package students avg_package students
cse 2019           1        2           0        0
    2020           3        4           0        0
    2021           5        6           0        0
    2022           7        8           0        0
ece 2019           9       10           0        0
    2020          11       12           0        0
    2021          13       14           0        0
    2022          15       16           0        0

In [11]:
# Swap level-0 index with level-0 column
swapped_df = df.stack(level=0, future_stack=True).unstack(level=0).swaplevel(axis=1)
# Sort by level-0 row index
sorted_df = swapped_df.sort_index(level=0)

# Print result
sorted_df

cse         ece      cse      ece
            avg_package avg_package students students
2019 delhi            1           9        2       10
     mumbai           0           0        0        0
2020 delhi            3          11        4       12
     mumbai           0           0        0        0
2021 delhi            5          13        6       14
     mumbai           0           0        0        0
2022 delhi            7          15        8       16
     mumbai           0           0        0        0

### `Q-2:` Covid Cases Data Set Problem
 Make a DataFrame Using both Covid Dataset. With Country name as Level-0 and Provinance/State as Level-1 Index and date, No of cases and No of Deaths as Columns.

In [21]:
import pandas as pd
import numpy as np

deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
confirm = pd.read_csv('time_series_covid19_confirmed_global.csv')

In [37]:
import pandas as pd

# Melt confirmed cases to long format
confirm_melt = confirm.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    var_name='date',
    value_name='No of cases'
)

# Melt deaths to long format
deaths_melt = deaths.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    var_name='date',
    value_name='No of Deaths'
)

# Merge the datasets
merged = confirm_melt[['Province/State', 'Country/Region', 'date', 'No of cases']].merge(
    deaths_melt[['Province/State', 'Country/Region', 'date', 'No of Deaths']],
    on=['Province/State', 'Country/Region', 'date'],
    how='outer'
)

# Convert date to datetime
merged['date'] = pd.to_datetime(merged['date'], format='%m/%d/%y')

# Handle missing Province/State
merged['Province/State'] = merged['Province/State'].fillna('Unknown')

# Set MultiIndex
merged.set_index(['Country/Region', 'Province/State', 'date'], inplace=True)

# Select required columns
result = merged[['No of cases', 'No of Deaths']].sort_index()

# Print result
result

No of cases  No of Deaths
Country/Region Province/State date                                 
Afghanistan    Unknown        2020-01-22            0             0
                              2020-01-23            0             0
                              2020-01-24            0             0
                              2020-01-25            0             0
                              2020-01-26            0             0
...                                               ...           ...
Zimbabwe       Unknown        2022-12-29       259981          5637
                              2022-12-30       259981          5637
                              2022-12-31       259981          5637
                              2023-01-01       259981          5637
                              2023-01-02       259981          5637

[311253 rows x 2 columns]

### `Q-3:` Show Country with Heighest death percent out of confirmed Cases.

In [35]:
# Aggregate total cases and deaths by Country/Region
country_totals = result.groupby(level='Country/Region').sum()

# Calculate death percentage (CFR)
country_totals['Death_Percentage'] = (country_totals['No of Deaths'] / country_totals['No of cases'] * 100).fillna(0)

# Find country with highest death percentage
highest_death_country = country_totals['Death_Percentage'].idxmax()
highest_death_value = country_totals['Death_Percentage'].max()

# Print result
print(f"Country with highest death percentage: {highest_death_country}")
print(f"Death percentage: {highest_death_value:.2f}%")

Country with highest death percentage: Korea, North
Death percentage: 600.00%


In [36]:
country_totals

,No of cases,No of Deaths,Death_Percentage
Country/Region,,,
Afghanistan,116218938,4901323,4.217319
Albania,163505982,2248012,1.374881
Algeria,164830894,4447129,2.697995
Andorra,21390392,116300,0.543702
Angola,53083822,1104387,2.080459
...,...,...,...
West Bank and Gaza,366130648,3377981,0.922616
Winter Olympics 2022,179152,0,0.000000
Yemen,7091065,1372952,19.361718


### `Q-4` : Make a dataframe for India from Covid Data with one extra column representing no of new cases.

* Just for Assumption "No of new cases" will be equal to difference of "no of cases" with previous day.
* First day new cases will be NaN or equal to no of cases

Say on 12/30/22 No of cases is - 44679608	and a day previous (12/29/22) no of cases is - 44679382.

Then for 12/30/22 -> No of New Cases  = 44679608 - 44679382 =

Note:- Try using shift Function

Try using the shift function

```
s = pd.Series([1,2,3,4,5,6])
s #-> [1,2,3,4,5,6]
s.shift(1) #-> [NaN, 1,2,3,4,5]
s.shift(-1) #-> [2,3,4,5,6, NaN]
```

In [42]:
# Filter for India
india_data = result.loc['India'].reset_index()

# Calculate No of new cases using shift
india_data['No of new cases'] = india_data['No of cases'] - india_data['No of cases'].shift(1)

# Select required columns
india_df = india_data[['date', 'No of cases', 'No of Deaths', 'No of new cases']]
india_df

,date,No of cases,No of Deaths,No of new cases
0,2020-01-22,0,0,NaN
1,2020-01-23,0,0,0.0
2,2020-01-24,0,0,0.0
3,2020-01-25,0,0,0.0
4,2020-01-26,0,0,0.0
...,...,...,...,...
1072,2022-12-29,44679382,530699,243.0
1073,2022-12-30,44679608,530702,226.0
1074,2022-12-31,44679873,530705,265.0
1075,2023-01-01,44680046,530707,173.0


### `Q-5:` Read the Dataset using the below given link and create a multi-index dataframe using the columns "Country" and "City/Town".

This dataset is about the most polluted cities in the world. You can get details from [here](https://www.kaggle.com/datasets/rajkumarpandey02/worlds-most-air-polluted-countries-cities).

**Dataset link:** https://tinyurl.com/2fe6vz4u Directly use this link to read.

**Task:**
1. Find out the name of the city of India which is most poluted based on PM10.
2. Find out the name of the city of India which has minumum pollution level based on PM10.
3. Do same operations (like 1 and 2) with the country China.
4. Make a pie chart based on the column "PM10" of the country Poland.
5. Make a bar chart based on the columns "PM2.5" and "PM10" of the countries Israel and Qatar.
6. Convert this MultiIndex DataFrame to Series by retaining the informations.

In [53]:
import pandas as pd
import matplotlib.pyplot as plt

# Read dataset
df = pd.read_csv("List of most-polluted cities by particulate matter concentration.csv")

# Create MultiIndex DataFrame
df_multi = df.set_index(['Country', 'City/Town'])
df_multi.drop(columns=['Unnamed: 0'], inplace=True)
# Task 1: Most polluted city in India based on PM10
india_most_polluted = df_multi.loc['India']['PM10'].idxmax()
india_max_pm10 = df_multi.loc['India']['PM10'].max()

# Task 2: Least polluted city in India based on PM10
india_least_polluted = df_multi.loc['India']['PM10'].idxmin()
india_min_pm10 = df_multi.loc['India']['PM10'].min()

# Task 3: Most and least polluted cities in China based on PM10
china_most_polluted = df_multi.loc['China']['PM10'].idxmax()
china_max_pm10 = df_multi.loc['China']['PM10'].max()
china_least_polluted = df_multi.loc['China']['PM10'].idxmin()
china_min_pm10 = df_multi.loc['China']['PM10'].min()

# Print results for India and China
print(f"Most polluted city in India: {india_most_polluted}, PM10: {india_max_pm10}")
print(f"Least polluted city in India: {india_least_polluted}, PM10: {india_min_pm10}")
print(f"Most polluted city in China: {china_most_polluted}, PM10: {china_max_pm10}")
print(f"Least polluted city in China: {china_least_polluted}, PM10: {china_min_pm10}")

import plotly.graph_objects as go
import plotly.express as px

# Task 4: Pie chart for PM10 in Poland
poland_pm10 = df_multi.loc['Poland']['PM10']
fig1 = go.Figure(data=[
    go.Pie(labels=poland_pm10.index, values=poland_pm10, textinfo='label+percent')
])
fig1.update_layout(
    title='PM10 Distribution in Poland Cities',
    template='plotly_dark'
)
fig1.show()

# Task 5: Bar chart for PM2.5 and PM10 in Israel and Qatar
israel_qatar = df_multi.loc[['Israel', 'Qatar']][['PM2.5', 'PM10']].reset_index()
fig2 = go.Figure(data=[
    go.Bar(name='PM2.5', x=israel_qatar['City/Town'], y=israel_qatar['PM2.5']),
    go.Bar(name='PM10', x=israel_qatar['City/Town'], y=israel_qatar['PM10'])
])
fig2.update_layout(
    title='PM2.5 and PM10 in Israel and Qatar',
    xaxis_title='City',
    yaxis_title='Concentration (µg/m³)',
    barmode='group',
    template='plotly_dark'
)
fig2.show()

# Task 6: Convert MultiIndex DataFrame to Series
series_data = df_multi.stack()
print(series_data.head())

Most polluted city in India: Kanpur, PM10: 319
Least polluted city in India: Aizawl, PM10: 44
Most polluted city in China: Jinan, PM10: 192
Least polluted city in China: Anshun, PM10: 38


Country  City/Town                   
India    Kanpur     Position                1
                    Year                 2016
                    PM2.5                 173
                    Temporal coverage    >75%
                    PM10                  319
dtype: object


In [54]:
df_multi

Position  Year  PM2.5 Temporal coverage  PM10  \
Country     City/Town                                                     
India       Kanpur               1  2016    173              >75%   319   
            Faridabad            2  2016    172              >75%   316   
            Gaya                 3  2016    149        50% -< 75%   275   
            Varanasi             4  2016    146              >75%   260   
            Patna                5  2016    144              >75%   266   
...                            ...   ...    ...               ...   ...   
Poland      Lublin             496  2016     27              >75%    31   
            Siedlce            497  2016     27              >75%    29   
            Trzebinia          498  2016     27              >75%    35   
El Salvador San Salvador       499  2016     27        50% -< 75%    59   
Turkey      Sivas              500  2016     27              >75%    56   

                         Temporal coverage.1  Database version (year)  
Country     City/Town                                                  
India       Kanpur                       NaN                     2018  
            Faridabad                    NaN                     2018  
            Gaya                         NaN                     2018  
            Varanasi                     NaN                     2018  
            Patna                        NaN                     2018  
...                                      ...                      ...  
Poland      Lublin                      >75%                     2018  
            Siedlce                     >75%                     2018  
            Trzebinia                   >75%                     2018  
El Salvador San Salvador                 NaN                     2018  
Turkey      Sivas                       >75%                     2018  

[500 rows x 7 columns]